We'll be using the [`RenderMan`](https://github.com/fedden/RenderMan) library to open VST plugins to create synthesized audio clips. After following the instructions to build the library, we need to ensure a few things:

- We need to change the python header search path from `/usr/include/python2.7` to  `/anaconda/envs/<conda_env>/lib/python2.7` in the `RenderMan.jucer` file in order to link to the conda version of python
- Per the installation instructions, we'll need to rename the built library `mv librenderman.so.dylib librenderman.so`, and move it to this library to use it.
- use `otool -L librenderman.so` to validate that we've linke to the correct version of python

In [1]:
import os
import librenderman as rm
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
from IPython.core.display import HTML
from tqdm import tqdm
%matplotlib inline

In [2]:
sampleRate = 44100
bufferSize = 512
fftSize = 512

engine = rm.RenderEngine(sampleRate, bufferSize, fftSize)
sylenth_path = '/Library/Audio/Plug-Ins/VST/Sylenth1.vst'

def load_plugin():
    if engine.load_plugin(sylenth_path):
        print('loaded plugin succesfully')
        
load_plugin()

loaded plugin succesfully


In [3]:
# Settings to play a note and extract data from the synth.
midiNote = 48
midiVelocity = 127
noteLength = 0.5
renderLength = 5.0

In [4]:
def load_preset(preset_path):
    if not engine.load_preset(preset_path):
        print('error loading preset...')

In [56]:
characteristics = {
    'envelope': {},
    'reverb': {},
    'delay': {},
    'space': {},
    'filter': {},
    'distortion': {},
}

# Amplitude Envelope characteristics
characteristics['envelope']['base'] = [
#     [0, 0.00], # AmpEnv A Attack
#     [1, 0.50], # AmpEnv A Decay
#     [2, 0.05], # AmpEnv A Release
#     [3, 0.50], # AmpEnv A Sustain    
#     [4, 0.00], # AmpEnv B Attack
#     [5, 0.50], # AmpEnv A Decay
#     [6, 0.05], # AmpEnv A Release
#     [7, 0.50], # AmpEnv A Sustain    
]

characteristics['envelope']['med_attack'] = [
    [0, 0.50], # AmpEnv A Attack   
    [4, 0.50], # AmpEnv B Attack
]

characteristics['envelope']['no_attack'] = [
    [0, 0.00], # AmpEnv A Release
    [4, 0.00], # AmpEnv B Release
]

characteristics['envelope']['med_release'] = [
    [2, 0.50], # AmpEnv A Release
    [6, 0.50], # AmpEnv B Release
]

characteristics['envelope']['no_release'] = [
    [2, 0.05], # AmpEnv A Release
    [6, 0.05], # AmpEnv B Release
]

# Space characteristics
characteristics['space']['no_reverb_no_delay'] = [
    [220, 0.00], # xSW DelayOnOff
    [224, 0.00], # xSW ReverbOnOff
]

# Reverb characteristics
characteristics['reverb']['base'] = [
    [224, 1.00], # xSW ReverbOnOff
    [142, 0.25], # Reverb Damp
    [143, 0.50], # Reverb Dry/Wet
    [144, 0.50], # Reverb Predelay
    [145, 0.50], # Reverb Size
    [146, 0.50], # Reverb Width    
]

characteristics['reverb']['big_reverb'] = [
    [145, 0.75], # Reverb Size
    [146, 0.75], # Reverb Width
]
characteristics['reverb']['small_reverb'] = [
    [145, 0.40], # Reverb Size
    [146, 0.40], # Reverb Width
]
characteristics['reverb']['no_reverb'] = [
    [224, 0.00], # xSW ReverbOnOff
]


# Delay characteristics
characteristics['delay']['base'] = [
    [220, 1.00], # xSW DelayOnOff
    [25, 0.5],   # Delay Dry/Wet
    [26, 0.5],   # Delay Feedback
    [27, 0.0],   # Delay HighCut
    [28, 0.0],   # Delay LowCut
    [29, 0.0],   # Delay PingPong
    [30, 0.0],   # Delay Smear
    [31, 0.5],   # Delay Spread
    [32, 0.8],   # Delay Time Left
    [33, 0.8],   # Delay Time Left
    [34, 0.5],   # Delay Time Left
]

characteristics['delay']['big_delay'] = [
    [25, 0.5],   # Delay Dry/Wet
    [26, 0.5],   # Delay Feedback
]
characteristics['delay']['small_delay'] = [
    [25, 0.25],  # Delay Dry/Wet
    [26, 0.25],  # Delay Feedback
]
characteristics['delay']['no_delay'] = [
    [220, 0.00], # xSW DelayOnOff
]

# Filter characteristics
characteristics['filter']['base'] = [
    [54, 0.5],  # FilterCtl Cutoff
]

characteristics['filter']['low_cutoff'] = [
    [54, 0.25],  # FilterCtl Cutoff 
]

characteristics['filter']['lo_mid_cutoff'] = [
    [54, 0.375],  # FilterCtl Cutoff
]

characteristics['filter']['mid_cutoff'] = [
    [54, 0.5],  # FilterCtl Cutoff
]

characteristics['filter']['mid_hi_cutoff'] = [
    [54, 0.675],  # FilterCtl Cutoff
]

characteristics['filter']['hi_cutoff'] = [
    [54, 0.75], # FilterCtl Cutoff 
]

# Distortion characteristics
characteristics['distortion']['base'] = [
    [221, 1.00], # xSW DistOnOff
    [35, 0.50],  # Distort Amount
    [36, 1.00],  # Distort DryWet
    [37, 0.00],  # Distort Type
]

characteristics['distortion']['on'] = [
    [221, 1.00], # xSW DistOnOff
]

characteristics['distortion']['off'] = [
    [221, 0.00], # xSW DistOnOff
]

In [57]:
def set_parameters(params):
    for [param, value] in params:
        engine.set_parameter(param, value)

In [58]:
entries = []
def isvalid(filename):
    return os.path.isfile(filename) and '.fxp' in filename

def makePresetEntry(dirpath, patch_name):
    path_name = os.path.join(dirpath, patch_name)
    patch_name = patch_name.replace(".fxp", "")
    patch_name = patch_name.replace(" ", "_")
    group = dirpath.replace("sylenth_patches/", "")
    entry =(path_name, patch_name, group)
    return entry
    
for (dirpath, dirnames, filenames) in os.walk('sylenth_patches'):
    for f in filenames:
        if not isvalid(os.path.join(dirpath, f)): continue
        entry = makePresetEntry(dirpath, f)
        entries.append(entry)

In [79]:
def render_audio(filename):
    def render(output_filename):
        engine.render_patch(midiNote, midiVelocity, noteLength, renderLength, False)
        audio = engine.get_audio_frames()
        audio = np.array(audio, np.float)

        librosa.output.write_wav(output_filename, audio, sampleRate)

        mp3_output_filename = output_filename.replace(".wav", ".mp3")
        os.system("ffmpeg -i " + output_filename + " -vn -ar 44100 -ac 1 -ab 64k -f mp3 " + mp3_output_filename)
        os.remove(output_filename)
    # There's an error in the way that renderman clears audioBuffers, so we need to write twice to ensure
    # that the correct audio buffer is actually written... fortunately the processing is fast enough
    render(filename.replace('.wav', '_dummy.wav'))
    render(filename)
    os.remove(filename.replace('.wav', '_dummy.mp3'))

In [90]:
def make_output_filename(patch_name, modifier, group):
    patch_name = patch_name.replace('&', 'and')
    patch_name = patch_name.replace("'", '')    
    directory = 'output/' + group + "/" + patch_name
    if not os.path.exists(directory):
        os.makedirs(directory)
    output_filename = directory + "/" + patch_name + modifier + '.wav'
    return output_filename

def render_preset(path, patch, group):
    # render preset basic
    load_preset(path)
    output_filename = make_output_filename(patch, "__base", group)
    
    render_audio(output_filename)
    
    # render preset variants
    for c in characteristics:
        for v in characteristics[c]:
            if v == "base": continue
            load_preset(path)
            modifier = '__' + c + '_' + v
            output_filename = make_output_filename(patch, modifier, group)
            
            if characteristics[c].get("base"):
                base = characteristics[c]["base"]
                set_parameters(base)
                            
            variant = characteristics[c][v]
            set_parameters(variant)            
            render_audio(output_filename)

for (path, patch, group) in tqdm(entries[285:]):
    render_preset(path, patch, group)

100%|██████████| 83/83 [07:51<00:00,  5.68s/it]
